In [10]:
!pip install transformers accelerate Pillow requests https://github.com/Dao-AILab/flash-attention/releases/download/v2.5.8/flash_attn-2.5.8+cu118torch2.3cxx11abiFALSE-cp311-cp311-linux_x86_64.whl
!apt install nvidia-cuda-toolkit --yes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.1/122.1 MB 8.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libaccinj64-11.5 libatk-wrapper-java
  libatk-wrapper-java-jni libbabeltrace1 libcub-dev libcublas11 libcublaslt11
  libcudart11.0 libcufft10 libcufftw10 libcuinj64-11.5 libcupti-dev
  libcupti-doc libcupti11.5 libcurand10 libcusolver11 libcusolvermg11
  libcusparse11 libdebuginfod-common libdebuginfod1 libegl-dev libgail-common
  libgail18 libgl-dev libgl1-mesa-dev libgles-dev libgles1 libglvnd-core-dev
  libglvnd-dev libglx-dev libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libipt2
  libnppc11 libnppial11 libnppicc11 libnppidei11 libnppif11 libnppig11
  libnppim11 libnppist11 libnppisu11 libnppitc11 libnpps11 libnvblas11
  libnvidia-compute-495 libnvidia-compute-510 libnvidia-compute-535
  libnvidia-ml-dev libnvjpeg11 libn

In [52]:
import requests
import torch
import torch.nn.functional as F
from PIL import Image
from io import BytesIO
from transformers import AutoModel, AutoTokenizer, AutoImageProcessor

def is_flash_attn_available():
    try:
        import flash_attn
        return True
    except ImportError:
        return False

@torch.inference_mode()
def load_model():
    use_optimized = torch.cuda.is_available() and is_flash_attn_available()

    model = AutoModel.from_pretrained(
        "visheratin/mexma-siglip2",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        optimized=False,
    )
    tokenizer = AutoTokenizer.from_pretrained("visheratin/mexma-siglip2")
    processor = AutoImageProcessor.from_pretrained("visheratin/mexma-siglip2")

    if torch.cuda.is_available():
        model = model.to("cuda")

    return model, tokenizer, processor

model, tokenizer, processor = load_model()
device = "cuda" if torch.cuda.is_available() else "cpu"


In [54]:
cache = dict()

def get_iiif_images(manifest_url):
    manifest = requests.get(manifest_url).json()
    results = []
    for canvas in manifest.get("items", []):
        try:
            img_url = canvas["items"][0]["items"][0]["body"]["id"].replace("full/full/0/default.jpg", "full/500,/0/default.jpg")
            canvas_id = canvas["id"]
            manifest_id = manifest_url
            results.append((img_url, canvas_id, manifest_id))
        except (KeyError, IndexError):
            continue
    return results[:5]

def classify_image(image: Image.Image, queries: list[str]):
    if image is None or not queries:
        return None

    if cache.get("text_inputs"):
        text_inputs = cache["text_inputs"]
    else:
        text_inputs = tokenizer(queries, return_tensors="pt", padding=True).to(device)
        cache["text_inputs"] = text_inputs

    processed_image = processor(images=image, return_tensors="pt")["pixel_values"]
    processed_image = processed_image.to(torch.bfloat16).to(device)

    with torch.inference_mode():
        image_logits, _ = model.get_logits(
            text_inputs["input_ids"],
            text_inputs["attention_mask"],
            processed_image
        )
        probs = F.softmax(image_logits, dim=-1)[0].cpu().tolist()

    return {queries[i]: probs[i] for i in range(len(queries))}


In [55]:
def process_manifest(manifest_url:str, queries:list[str]):
    if not queries:
        raise ValueError("No valid queries provided.")

    image_canvas_manifest = get_iiif_images(manifest_url)
    print(f"Found {len(image_canvas_manifest)} images in the manifest.")

    results = []
    for i, (image_url, canvas_id, manifest_id) in enumerate(image_canvas_manifest):
        print(f"\nProcessing image {i+1}/{len(image_canvas_manifest)}: {image_url}")
        try:
            response = requests.get(image_url)
            image = Image.open(BytesIO(response.content)).convert("RGB")
            classification = classify_image(image, queries)
            results.append((image_url, classification))
        except Exception as e:
            print(f"Failed to process image: {e}")
            results.append((image_url, {"error": str(e)}))
    return results


In [56]:
terms = ["labels","uittredingen (van staten)","plof- en ramkraken ram- en plofkraken ramkraken","migratiecrises vluchtelingencrises","lgbti's lhbtiq+'s queer queers","fake news nepnieuws","cryptogeld cryptovaluta virtueel geld","dark web","VAR videoscheidsrechters","e-sports electronic sports","waarschuwingscodes weeralarm","vleesvervangers","elektrische bakfietsen elektrische bolderkarren elektrische vervoermiddelen elektrische voertuigen","diervrij vlees in-vitrovlees kweekvlees laboratoriumvlees namaakvlees nepvlees","gehoorschade oorsuizen tinnitus","e-waste elektronisch afval","stereotypering","cookies","klimaatbeleid klimaatpolitiek","groenteboeren","asielrecht kinderpardon","MMA mixed martial arts","dividendbelasting opbrengstbelasting","energielabels","wifi","werkdruk","AR augmented reality toegevoegde realiteit","gemengde werkelijkheid mixed reality","antidepressiva antidepressivum stemmingsverbeteraars","daten dating","biomassa","asielzoekerscentra azc's","lyofilisatie vriesdrogen","DAB DAB+ digitale radio","crystal meth methamfetamine","handelsconflicten handelsoorlogen","inkomensverdeling","pijnstillers","mediawijsheid","2019-nCoV COVID-19 MERS-CoV Omicron SARS-CoV-2-virus Wuhan-virus corona coronavirus","pandemie\u00ebn","sociaal isolement sociale isolatie","mondkapjes","overdosering overdosis","homohuwelijk homohuwelijken","diversiteitsbeleid inclusief beleid","imperialisme","virginity maagdelijkheid maagden","macho culture machocultuur","economisch beleid","haken","keizerinnen","genderidentiteit","overgang","wetenschappelijke onderzoekers wetenschappers","neurodiversiteit","FTM female-to-male transmannen vrouw-naar-man","MTF male-to-female man-naar-vrouw transvrouwen","drag dragkings dragqueens","windfoilen","telekinese","luisterboeken","verkeerspolitie","tolerantie verdraagzaamheid","ayurveda","milieubewustzijn","laster smaad","jakhalzen","bedreigde plantensoorten","ontpolderingen","bezetting","pensioenleeftijd pensionering","verkeersregelaars","canons","afkicken","digitale televisiekanalen themakanalen","striptease","koopzondagen","dierenactivisme dierenrechtenactivisme","bonusregelingen","constructivisme","levensloop","stamcellen","paaldansen","emissierechten","complottheorie\u00ebn","Q-koorts","garagebedrijven","gebitten tanden","flirten","zorgboerderijen","geldautomaten","hennepteelt wietteelt","broeikasgassen koolstofdioxide","champagne","eerlijke handel fairtrade","3D-techniek","burgerinitiatieven","bodyscanners","biodiversiteit","aprilgrappen","lama's","holtedieren neteldieren","ecoducten natuurbruggen","homomanifestaties lhbtiq+-manifestaties","vingerafdrukken","vioolbouwers","creationisme schepping","stemcomputers","biometrie","toeters vuvuzela's","boerka's chadors islamitische kleding nikabs","afvalverbranding vuilverbranding","e-bikes e-fietsen elektrische fietsen","ziekte van Huntington","kernsplijting kernsplitsing","patates frites","eikenprocessierupsen","evenementen manifestaties (evenementen)","loopbaanontwikkeling","exoten (flora en fauna)","uitgestorven diersoorten","reanimatie","stadswachten","sociaalnetwerksites social media sociale media sociale netwerksites","helpdesks","autogas lpg","bekeuringen boetes","nanotechnologie","militaire terugtrekkingen","afscheid zwanenzang","speciale effecten","resocialisatie","nationaliteit staatsburgerschap","onschendbaarheid","actiegroepen","neerslag","debatten","heimwee","prenataal onderzoek","straattaal","dansfilm","crashed ice schaatscross","Gilles-de-la-Tourette-syndroom syndroom van Gilles de la Tourette","nierdialyse","zalmen","sportcuriosa","automutilatie zelfverwondingen","tankvliegtuigen","populisme","waterkeringen","partijleiders","autodealers autohandel","toneelgezelschappen toneelgroepen","overheidsberoepen","buiksprekers","vogelonderzoek","audioboodschappen radioboodschappen spraakberichten voicememo's","dodo's","kredietbeoordeling","gsm's (smartphones) smartphones","schaliegas","snelheidsbeperkingen snelheidslimieten","carbidschieten","vogelkooien voli\u00e8res","syndroom van M\u00fcnchhausen by proxy","e-books","vliegeren","natuurijs","beledigingen","incassobureaus","zonnestormen zonnewinden","raketschilden ruimteschilden","contactverbod gebiedsverbod straatverbod","eksters","kiteboarden kitesurfen","gewetensbezwaarden weigerambtenaren","beeldvorming imago reputatie","webcamera's webcams","identiteit individualiteit persoonlijkheid","oliebollen","gigolo's","elektroschokwapens stroomstootwapens tasers","therapeuten","gearrangeerde huwelijken gedwongen huwelijken huwelijksdwang uithuwelijking","wrakingen","duurzaamheid","televisie-innovaties","prostaatkanker","bbl beroepsbegeleidende leerweg","astero\u00efden planeto\u00efden","blunders flaters","affaires schandalen","koeriersbedrijven","medisch tuchtrecht tuchtrecht","grafeen","drugshandel","illegale handel zwarte handel","seksartikelen sextoys slaapkamerspeeltjes","patrijzen veldhoenders","oppositie","3D-printers","commissarissen van de koning","spinnen (textiel)","dierenvoer diervoeders","crowdfunding fondsenwerving","huizenmarkt woningmarkt","hoogtezonnen solaria zonnebanken zonnestudio's","verdrinkingsdood","plankzeilen surfen","e-sigaretten elektronische sigaretten shisha-pennen","obesitas vetzucht","spoedeisende hulp","gebruikte goederen kringloopwinkels tweedehands goederen","parfums","baarden gezichtsbeharing snorren","sponsdieren sponzen","colportage","taboes","scootmobielen","airconditioning klimaatbeheersing luchtbeheersing","luchtverversing ventilatie","alevieten","arbeidsmigratie","auto-onderdelen autoaccessoires","blogs video weblogs vlogs weblogs","elektronische detentie enkelbanden","lichthinder lichtvervuiling","cheerleaders","balletgezelschappen dansgezelschappen dansgroepen","vrouwenprogramma's","kijkcijfers","jezidi's yezidi's","mensport paarden mennen","adolescenten jongvolwassenen","deeleconomie","doe-het-zelvers klussers","uitzendapparatuur","belichting","breedbeeld tv breedbeeldtelevisie","opnameapparatuur","ondertiteling ondertitels","baptisten","barbecues","barok","batterijen","bavianen","bazaars","beademing","beatniks","bedevaarten hadj","bedoe\u00efenen woestijnbewoners","bedreigde diersoorten","bedrijfsartsen","bedrijfskunde","bedrijfspanden","bedrijfssluitingen","bedrijfsspionage","bedrijfsverenigingen","beeldhouwers","beelden beeldhouwwerken borstbeelden bustes sculpturen","begijnhoven","begrafenisondernemers doodgravers","beiaardiers","bejaardenverzorgers geriatrisch zorgmedewerkers","bekeringen","beleggingen","beren","beroepskeuze","beroepsziekten","berovingen roof zakkenrollerij","beschietingen","besloten vennootschappen bv's","besnijdenissen","bessen cranberry's","bestrijdingsmiddelen gif herbiciden insecticiden pesticiden","beton","beuken","beulen","beurscrash beurskrach","beurshandelaren hoekmannen","beurzen handelsbeurzen jaarbeurzen vakbeurzen","beveiliging","bevers","bevolkingsvraagstukken demografie overbevolking wereldbevolking","bevruchtingen","bewapening wapenwedloop","bewegingsleer","bewegingskunst bewegingstheater","bezuinigingen","biatlon","bibliotheken","biecht","bier","bierbrouwerijen","biggen","Bijbelstudies","bijen hommels","bijenteelt honingraten","bijgeloof","bijscholing","binnenhuisarchitectuur","biochemie","biogas","biografie\u00ebn necrologie\u00ebn","biologie","biologisch-dynamische landbouw biologische landbouw","bacteriologische wapens biologische wapens","biotechnologie","biseksualiteit","bisschoppen","bizons","blaasinstrumenten","bladmuziek noten (muziek) partituren","blindengeleidehonden","blindeninstituten","bloed","bloedbanken","bloedtransfusies","bloedwraak vendetta","bloemencorso's","bloemenveilingen","bloemschikken bloemsierkunst","blues","bobslee\u00ebn","bodemdaling","bodybuilding","boekbinders","boekdrukkunst","boekhandel boekwinkels","boetseren","bloembollenvelden bollenvelden","bommeldingen","bont","boodschappen","boogschieten handboogschieten","bookmakers","booreilanden","boortorens","bordelen seksclubs","borstkanker","boslandcreolen marrons","boswachters","arboretums botanische tuinen hortus botanicus","botulisme","boulevards","boulimia nervosa eetverslaving","bouwvakkers","bowling","boycots","braille","brandblussers","branden","brandkasten kluizen","brandpreventie","brandstichting pyromanen","brandweerkazernes","brandwonden","breien","broedmachines","broeikaseffect","bruiloften","buffels","arbeidsmigranten buitenlandse werknemers gastarbeiders","buizerds","bulldozers","bungeejumping","bunzings","bureaucratie","burgerlijke ongehoorzaamheid","burgeroorlogen stammenoorlogen","buschauffeurs","busstations","buurthuizen clubhuizen","buurtpreventie","cacao","cactussen","adelborsten cadetten","calvinisme","cameralieden","capitulaties overgave","astma bronchitis cara","caravans","cardiologie","beiaards carillons klokkenspel","carpooling","cartografie","casino's","cassave tapioca","castraties","catacomben","catamarans","catechismussen","cateringbedrijven","Guinese biggetjes cavia's","cd-roms","celibaat","cello's","cement","champignons","chansons","afpersing chantage","Noord Indiaas Klassiek chauffeurs","chemie scheikunde","chemisch afval","chemische stoffen","chemische wapens gifgassen","chimpansees","chirurgen","chloor","chocolade","cholera","christelijk onderwijs","christenen","chronische ziekten","chrysanten","cipiers gevangenbewaarders","circuits","civiele verdediging","clowns","coca","coca\u00efne crack","coffeeshops","collaboratie landverraad","collages","coma's","combines","commercialisering","commerci\u00eble omroep","commissarissen van de koningin","communicatie","communicatiesystemen","communie","compost","computernetwerken","computergames consolegames games","computervirussen","concentratiekampen vernietigingskampen","concert concerten","concertzalen muziekcentra","condooms","architecten","architectuur bouwkunst","artillerie","ouders","basisonderwijs lager onderwijs primair onderwijs","middelbare scholen secundair onderwijs voortgezet onderwijs","dood","presidenten","spoorwegen","vrijheid","artsen doktoren heelmeesters medici","petrochemische industrie","landschappen","junkies verslaafden","organisaties","clubs verenigingen","milieuverontreiniging","water","drinkwater","grondwater","bodemverontreiniging","luchtverontreiniging luchtvervuiling smog","milieubeheer milieubescherming","aardgaswinning gaswinning","aardolie oli\u00ebn petroleum","aardoliewinning olieboringen","accijnzen","acteurs actrices","adel","revoluties","rhythm & blues","lintjesregen ridderorden","ridders","rietdekkers","mandenvlechters mattenvlechters rietvlechters","rijexamens","rijkdom","rijnaken","robben","robots","rock-'n-roll","juicekanaal juicekanalen paparazzi roddelpers","rodelen","rodeo's","roken","rollerskating rolschaatsen","romantiek","roofdieren","roofvogels","rook","katholicisme rooms-katholicisme","rotan","rotondes","rotsen","rotstekeningen","rozen","RSI muisarm repetitive strain injury","rubberboten","rubbertappers","rugby","ruilverkaveling","planologie ruimtelijke ordening","ruimteveren","rupsen","sabotage","sadomasochisme","salamanders","panorama's","parachutes","opleidingen trainingen","loodsen magazijnen opslagplaatsen","educatie opvoeding","ornamenten","oorlogsveteranen oud-strijders","openbaar ambt overheidsdiensten rijksdiensten","dijkdoorbraken overstromingen watersnood","delicten overtredingen","parachutisten","pati\u00ebnten zieken","patrouilles","patrouillevaartuigen","personenvervoer","pest","pijpleidingen smeerpijpen","pastoors","helderziendheid parapsychologie telepathie","parkeerplaatsen parkeerterreinen","parken stadsparken","parlementsgebouwen","mailboten passagiersschepen","consumptieoli\u00ebn margarine oli\u00ebn plantaardige oli\u00ebn en vetten plantaardige vetten spijsoli\u00ebn","olijfolie","omheiningen","onafhankelijkheidsstrijd","onderhoud","officieren","gezondheidszorg volksgezondheid","handel","gezinnen","cultuurbeleid","aardappelen","aardbevingen","baby's zuigelingen","biljarten","binnenvaart","binnenvaartschepen","bioscopen theaters","bliksem","blinden","bloembollen","bloembollenteelt bollenteelt","bloemen","bevelhebbers","bevloeiingswerken irrigatie","bevolkingsgroepen","bevoorrading","bevrijding","bewaking","bloementeelt","bommenwerpers","bondgenoten","boomgaarden","bosbouw","boter","bouwkunde","bommen","aalscholvers","aangeboren aandoeningen aangeboren afwijkingen","aanslagen bomaanslagen moordaanslagen","aartsbisschoppen","abonneeradio betaalradio","abonneetelevisie betaaltelevisie","abortusklinieken","academici","aandachtswijk achterbuurten achterstandsbuurten herstructureringswijk krachtwijk prioriteitswijk probleemwijk wijk met een uitdaging","acrobatiek","actualiteitenprogramma's","acupunctuur","administratief personeel boekhouders kantoorpersoneel secretaressen typistes","advocaten","aerodynamica windtunnels","afasie","aanplakbiljetten affiches posters","afgravingen","afluisterapparatuur","afluisteren","afval huisvuil vuilnis","afvallozingen dumping","afvalverwerking vuilverwerking","aids","akkerbouw ploegen","akoestiek geluidsleer","alarmnummers","albinisme albino's","alcohol","alcoholisme alcoholverslaving drankzucht","algen wieren zeewier","alleenstaanden singles vrijgezellen","alternatieve straffen leerstraffen taakstraffen werkstraffen","amateurs","ambassades","American football","amfibie\u00ebn","amfitheaters","amnestie gratieverlening","amputaties","analfabetisme ongeletterdheid","analisten laboranten","anesthesie narcose","anorexia nervosa","antibiotica penicilline","antiek antiquiteiten","antilopen","antiquairs","antisemitisme","antropologie culturele antropologie etnografie etnologie volkenkunde","antroposofie","apartheid","apen","apocalyptiek einde van de wereld","apotheken apotheker apothekers farmacie","appels","aquarellen","aquaria","arbeidsbemiddeling banenpools","arbeidsbureaus","arbeidsinspectie","arbeidsomstandigheden werkomstandigheden","arbeidsongeschiktheid","arbeidsongevallen bedrijfsongevallen","archeologie oudheidkunde","archieven dossiers gemeentearchieven rijksarchieven","adelaars arenden","armoede","aanhoudingen arrestaties","asbest","asfalt","alleenstaande minderjarige asielzoekers asielzoekers","asperges","astrologie horoscopen","astrofysica astronomie kometen sterrenkunde","athe\u00efsme vrijdenkerij","au pairs kindermeisjes nanny","audities","autisme","autobiografie\u00ebn egodocumenten memoires","auto-industrie autofabrieken","automatisering","autonomie zelfbestuur","autorijscholen rijinstructeurs","autosloperijen","autosport","autotests","baarmoederhalskanker","bacteri\u00ebn","badhuizen sauna's","badminton","baileybruggen","bakens boeien","bakfietsen","bakkerijen bakkers banketbakkers","baksteenindustrie steenfabrieken","ballen","dansopleidingen dansscholen","ballingschap verbanningen","ballonnen","ballonvaart","bals","balsporten","bamboe","bananen","condors","conducteurs","confucianisme","conservatisme","conservatoria","conserven","conserveringsmiddelen","consulaten","consultatiebureaus","consumenten","containers","controlekamers","corruptie omkoping","cosmetica make-up","country & western","couveuses","cowboys ranchmedewerker","crematoria","creolen","cricket","cruiseschepen","culturele identiteit","curling","cursussen","dada\u00efsme","dagboeken","dakbedekking","daklozen zwervers","dansen","balletdansers danseressen dansers","darts","dassen","databanken","decorontwerpers","decors","deeltijdarbeid","deeltjesversnellers versnellingsmachines","dekolonisatie","dementie ziekte van Alzheimer","betogingen demonstraties (betogingen) klimaatmarsen manifestaties (betogingen) protesten protestmarsen protestoptochten vredesdemonstraties","depressies","derwisjen","deserties","destructiebedrijven","detectives","detectors","deurwaarders","devaluatie","diagnostiek","dialecten","diamantbewerkers diamantindustrie","dichters","di\u00ebten","dienstplicht militaire dienst","dienstweigeraars","dierenartsen","dierenasielen","dierenbescherming","dierenklinieken","dierenmishandeling","dierenverzorging","diergeneeskunde","di\u00ebtisten","digitale techniek","dijkverzwaringen","dinosaurussen","dioxine","diploma's getuigschriften","diplomatie diplomatieke dienst","directeuren","diskjockeys","discriminatie","dissidenten","distilleerderijen stokerijen","distributie","distributiebedrijven","dixieland","docudrama's","documentaires","dolfijnen","dolfinaria","dompteurs leeuwentemmers","bloeddonors donors orgaandonaties","dooi","doolhoven","doopsgezinden mennonieten","amfetamine anabole stero\u00efden doping epo","douane","doveninstituten","doventolken","draagmoederschap","draaiorgels pierementen","drainage","draken","drankbestrijding drankverbod","drenkelingen","cfk's drijfgassen","drogisterijen","dromedarissen","dromen","drugsbeleid","druiven","drukkerijen offsetdruk zetterijen","drukkers","drukpersen","duiksport onderwatersport","duivels","duiven","duivensport","motorbendes motorclubs","duwvaart","Arbeitseinsatz arbeidsinzet dwangarbeid","dwarslaesie","echo's echoscopie","ecologie","economie economische wetenschappen","edelherten","edelsmeden goudsmeden zilversmeden","eekhoorns","schichters streakers","eenden","eenoudergezinnen ongehuwde moeders","eenzaamheid","effectenbeurzen","waarnemers","egels","egyptologie","eieren","eilanden","voedselveiligheid","eiwitten enzymen","elanden","trendwatchen trendwatchers","elektrische auto's","debuten","desktoppublishing elektronische beeldverwerking","primeurs scoops","elektronische muziek synthesizers","aanbestedingen","elektrotechniek","gamefixing matchfixing wedstrijdvervalsing","bevolkingskrimp demografische krimp krimpgebieden krimpregio's","emancipatie","emblemen","embryo's","emigranten","emigratie","encyclieken","streetdance","encyclopedie\u00ebn","energiebedrijven","energiebesparing","energiecrises","epilepsie","ereburgerschappen","erewachten","erfelijke ziekten","erfenissen","erfrecht","ergonomie","erosie","erotiek","etalages","ethiek moraal","ethologie","etiquette","etnische conflicten rassenstrijd stammenoorlogen","allochtonen etnische minderheden minderheden","etnische zuiveringen","etsen","euthanasie","evacuaties","reisadviezen","evacu\u00e9s","evangelisatie","evenaar","evolutie","eindexamens examens","executies terechtstellingen","exhibitionisme","duiveluitdrijvingen exorcisme","experimentele kunst","experimenten proefnemingen simulaties","explosies ontploffingen","export","expressionisme","rechts-extremisme","ezels","fabels","fado's","brassbands fanfares","fans","fantasie","farmaceutische industrie geneesmiddelenindustrie","fastfood","fauna","fazanten","feestartikelen","festivals","fietscross","fietsenmakers","fietsenstallingen","fietstaxi's riksja's","figuranten","filatelie","filmconservering","filmindustrie","filmkeuringen","filmmontage","filmoperateurs","filosofie wijsbegeerte","flamingo's","flora","blokfluiten dwarsfluiten fluiten","fluor","fobie\u00ebn","foetussen","fokkerijen","folk","forellen","forenzen woon-werkverkeer","fossielen","fotomodellen mannequins modellen","fouilleren","fraude verduisteringen witwassen","frisdranken","fundamentalisme","fusies","futen","futurisme","futurologie toekomstonderzoek","bewegingstherapie fysiotherapie heilgymnastiek manuele therapie mensendieck","galerie\u00ebn","ganzen","garnalen","culinaire zaken gastronomie kookkunst","gebaren","gebarentaal","bidden gebeden","geboorteregeling gezinsplanning","gedragsonderzoek","aalmoezeniers geestelijk verzorgers legerpredikanten","geesten","geestesziekten meervoudige persoonlijkheidsstoornissen psychische stoornissen","gehoorgestoorden slechthorenden","gehoororganen oren","geisers","geiten","audiocassettes cassettebandjes geluidsbanden","geluidsbarri\u00e8re","geluidshinder geluidsoverlast lawaai","geluidstechnici","genealogie geslachtkunde stambomen","generaals","erfelijkheidsleer genetica","genetische manipulatie genetische modificatie gentechnologie","gerechtsgebouwen paleizen van justitie","geschillencommissies","geslachtsorganen","chlamydia geslachtsziekten seksueel overdraagbare aandoeningen soa","eb getijden vloed","getto's","geuren reuk","gevaarlijke stoffen gif giftige stoffen","gevangenkampen strafkampen","gevonden voorwerpen","gewichtheffen","gewichtloosheid","gibbons","gieren","gieterijen","gilden schutterijen","giraffes","gitaar gitaren","gladheid","glas-in-lood","glasblazers","glaskunst","glasvezelkabels","glazenwassers","gletsjers","gnoes wildebeesten","catechisatie godsdienstonderwijs zondagsscholen","godsdienstvrijheid","golf","gorilla's","gospels","gotiek","goudvissen","goudzoekers","gouverneurs-generaal","gouverneurs","draglines graafmachines","etsers grafisch kunstenaars","grafische industrie","grafschennis","grammatica","gregoriaanse muziek","grensbewaking","Mexicaanse griep griep influenza","grimeren schminken","grind","groeihormonen","groenteveilingen","groenvoorziening plantsoenen","grondbeleid landhervormingen","grootgrondbezit","groothandel","grotonderzoek speleologie","grutto's","gymnasia","gynaecologie","haaien","haar","haaruitval","hagedissen","hagel","hamsters","handbal zaalhandbal","handschrift","hardware","haringen","harmonieorkesten","harpen","hart- en vaatziekten hartinfarcten trombose","hasj","haviken","havo hoger algemeen voortgezet onderwijs","hazen","hbo hoger beroepsonderwijs","hedendaagse muziek minimal music seri\u00eble muziek","heelal","heien","heiligdommen","heiligenbeelden","heiligverklaringen","heksen","heling","helmen","helmgras","hemofilie","hengelsport sportvissen","hennep","hepatitis leverontsteking","herdershonden","hero\u00efne","hersenen","herten","hi\u00ebrogliefen","hiphop","hitparades","hittegolven","hobo's","hoefsmeden","hogesnelheidstreinen tgv","holisme","holografie hologrammen","homeopathie","honden","hondenrennen","hondsdolheid rabi\u00ebs","hongerstakingen","hoeden hoofddeksels petten","hoofdpijn migraine","hoogbegaafdheid wonderkinderen","hooggerechtshoven","elektriciteitsmasten hoogspanningsmasten","hooien","forums hearings hoorzittingen","hormonen","griezelfilms horrorfilms","house","houthakkers","houthandel","houtindustrie","houtsnijwerk","hoveniers tuinmannen","hovercrafts luchtkussenvoertuigen","huid","huiden","huidkanker","huidaandoeningen","huifkarren","huisarrest","huisartsen","huishoudbeurzen","huishoudelijke apparatuur witgoed","huisnijverheid","huisuitzettingen","huiswerk","huiszoekingen","huizen van bewaring","hulpacties","hulpdiensten","humanisme","humor moppen","huurlingen","huurmoordenaars","huurwaardeforfaits","hyacinten","hyena's","hypnose hypnotherapie","hypotheken","iconen","iglo's","ijken","ijsberen","ijsbrekers","ijsboer ijscomannen ijsverkopers","ijsdansen","ijsspeedway","ijsvogels","ijszeilen","ijzel","illegalen ongedocumenteerden","imams","imitators","imkers","immigranten","impala's","import","impotentie","impressionisme","inbraak","incest","indianen","individualisering","industri\u00eble archeologie","infiltratie","geldontwaarding inflatie","informatica","infrarode straling infrarood","infrastructuur","ingenieurs","inktvissen octopussen","innovaties","inspici\u00ebnten","instortingen","instrumenten","instrumentmakers","inburgering integratie","intelligentie","intensive care","cd-i's interactieve media","internaten kostscholen schippersinternaten","internationale scholen","investeringen","IQ intelligentiequoti\u00ebnt","islamitisch onderwijs","ivoor","jaarmarkten","jaarverslagen","Nieuwjaar jaarwisselingen oudjaar","cheeta's jachtluipaarden","jachtopzieners","jagers","jaguars","bebop jazz","jazzballet","jetski's","jeugd- en zedenpolitie","jodelen","jeugdcentra jongerencentra","jongleurs","joyriding","judo","art nouveau jugendstil","jukeboxen","juweliers","kaalheid","kaasmakerijen","kaasmarkten","kabels","kabinetsformaties","kabouters","kalenders","kalligrafie schoonschrijven","kalveren","kamermuziek","kamerplanten","kamerverhuur","kanker","kannibalisme","kajaks kano's","kano\u00ebn","kantklossen","kantoorartikelen","kapellen","kapitalisme","karaoke","karavanen","kardinalen","karton","kartonindustrie strokartonfabrieken","katoen","kelders","kerkscheuringen schisma's","kerncentrales","kernfusie","kernproeven","kerstbomen kerstboom","kerstspelen","kerstversieringen","keurmerken","kibboetsen","kijk- en luisteronderzoek","kinderbescherming","kinderbijslag","kindermishandeling","buitenschoolse opvang kinderopvang","kinderpsychiatrie","kindertehuizen","kinderwagens","kinderziekten","kiosken","kitsch","klassieke muziek","confectie-ateliers confectie-industrie kledingindustrie","klei","kleinkunstacademies","kleuren","kleurstoffen","kleuteronderwijs","klokkengieterijen","klompen","klonen","kloosterorden","knaagdieren","knipkunst","knutselen","koffiebranderijen","koffiehuizen","kolenmijnen","kolonialisme","koninklijke bezoeken","konvooien","koopkracht","fotokopie\u00ebn kopie\u00ebn","koraalriffen","kostuums","kraamklinieken","kraamzorg","kraakbeweging krakers","krijgsraden militaire rechtbanken","kristallen","geneeskrachtige kruiden kruiden","kruisraketten","kruistochten","kubisme","kunstacademies","AI kunstmatige intelligentie","kunstmest","kunstnijverheid","kunstorganen","artotheken kunstuitleen","kunstvezels nylon","kunstzinnige vorming","kurk","kussen","kustvaarders","kustwacht","kwakzalvers","kwelders","lachen","landbouwco\u00f6peraties","landbouwonderwijs","landbouwuniversiteiten","havezaten landgoederen","geodesie landmeetkunde","landschapsparken natuurparken","aardverschuivingen lawines","lager beroepsonderwijs lbo","herkenningsmelodie\u00ebn leaders","leasing","lederwaren","leegstand","leer","leerlingwezen","leeuwen","mayonaise sauzen","leguanen","kredieten leningen microkredieten","lepelaars","lepra melaatsheid","lerarenopleidingen","collegegelden lesgelden schoolgelden","bloedkanker leukemie","levensverzekeringen","leesvaardigheid lezen","libellen","licht","lichtschepen","liederen","liefde","lifters","lijsters","leilinden linden","liplezen","lithografie\u00ebn steendruk","lobbyen","logistiek","logo's vignetten","logopedie spraaklessen","lokale omroepen","loodgieters","loodsboten","lsd","bombardementen luchtaanvallen","luchtmobiele brigade","luchtschepen zeppelins","luchtvaartmaatschappijen","luchtverkeersleiders","luchtverkenning","bladluis hoofdluis luizen","lynxen","maanlandingen","eclipsen maansverduisteringen","machinefabrieken","machinisten","maffia","magie","magneten magnetisme","ma\u00efs","majorettes","maki's","malaria","mammoeten","managementopleidingen","managers","manuscripten","mao\u00efsme","halve marathon marathon","marathonschaatsen","marconisten radiotelegrafie","marihuana wiet","marketing verkoopmethoden","marmotten","marsmuziek","martelingen","maskers","massacommunicatie","massage","massagraven","massamoorden","masturbatie","graftombes mausoleums","mavo middelbaar algemeen voortgezet onderwijs","mbo middelbaar beroepsonderwijs","mechanica","mechanisatie","inspraak medezeggenschap","media","mediabeleid omroeppolitiek","mediatraining","medicijnmannen traditionele genezers","bevolkingsonderzoek endoscopie medisch onderzoek scan-onderzoek","medisch specialisten specialisten","medische apparatuur pacemakers","medische ethiek","meditatie","meel","meeuwen","meineed","massa menigte mensenmassa","menopauze","merchandising","merels","meridianen","bemesting gier mest mineralen","mestverwerking","lassen metaalbewerking","metaalindustrie","meteorieten","meteorologie weerkunde","metselaars","meubelmakers","koolmezen mezen","micro-organismen","microfiches microfilm microkopie\u00ebn","microscopen","middeleeuwse muziek","midden- en kleinbedrijf middenstand","midgetgolf","mieren","migratie remigratie transmigratie","mijnwerkers","energieheffingen milieuheffingen","militaire academies","bajonetvechten exercities gevechtstrainingen militaire oefeningen","militaire politie","microfoons","intercoms","eventing military","milities","miljonairs","antrax miltvuur","mime pantomime","minderhedenbeleid","miniaturen","minimumloon","misdaadpreventie","detectiveromans misdaadromans","missionarissen","mode","modehuizen","modellenbureaus","modeontwerpers","moderne dans","moderne kunst","mannequins modeshows","moestuinen","moleculen","molenaars","mollen","MKZ mond-en-klauwzeer","harmonica's mondharmonica's","monsters","montessorionderwijs","automonteurs monteurs onderhoudspersoneel","moordaanslag moordaanslagen moorden","mortuaria","moslims","mosselen","mossen","motorboten","motorcross","motorsport","moza\u00efeken","muggen","muilezels","muiterij","muizen","multinationals","MS multiple sclerose","muntstelsels","muren","muskusratten","mussen","muziekhandel platenzaken","muziekinstrumenten","drumbands muziekkapellen muziekkorpsen","ME (syndroom) chronisch vermoeidheidssyndroom myalgische encefalomyelitis vermoeidheidssyndroom","mystiek","mytylscholen","naaisters","naamloze vennootschappen nv's","nachtclubs","narcissen","nasynchronisatie","naturalisatie","natuur","natuurparken natuurreservaten wildreservaten","kompassen navigatie","necrofilie","nederzettingen","negers zwarten","nertsen","nesten","neurologie","neurosen","neushoorns","nevenfuncties","nieuwbouw","nijlpaarden","nomaden","noodlandingen","noodwoningen","noorderlicht","jeugdsentiment nostalgie","notarissen","nozems","naaktstranden naturisme nudisme","kentekens nummerborden","oasen","occultisme","diepzee-onderzoek oceanen oceanografie zeebodemonderzoek","oecumene","oesters","officieren van justitie","okapi's","oldtimers","oliecrises","olifanten","olijven","ombudsmannen","omgangsregelingen","kijk- en luistergeld omroepbijdragen","omroepers omroepsters","omscholing","oncologie","onderduikers","ondernemers zakenlieden","ondernemingsraden","ondervoeding","ongediertebestrijding","onkruid","onroerende goederen","ontbossing","ontgroening","ontruimingen","ontslagen","ontsnappingen","kidnapping ontvoeringen","ontwapening","oogheelkunde","oogsten","oogstfeesten","maraboes ooievaars","oosterse kerken orthodoxe kerken","opium","opslagtanks","opticiens","optiebeurzen","optochten parades ticker-tape","opwerkingsfabrieken","orang-oetans","oratoria","orchidee\u00ebn","orgaanhandel","organen","orgelbouwers","orgeldraaiers","kerkorgels orgels","orka's","cyclonen orkanen","orkesten salonorkesten symfonieorkesten","orthopedie","otters","bejaarden ouderen senioren","ouderschap","ouderschapsverlof zwangerschapsverlof","magnetrons ovens","canyoning overlevingstochten survival","paleontologie","palingen","papier","gebedsgenezing handoplegging magnetiseurs paranormale geneeswijzen","parels","parkeergarages","parkeerwachten","parlementaire enqu\u00eates","Kamerleden parlementari\u00ebrs","passiespelen","pastoraal werk","patriottisme vaderlandsliefde","pauwen","grooming pedofilie","pelgrims","pelikanen","pelsdieren","pensioenfondsen","pensions","peren","persbureaus","persfotografen","persiflages","arbeidsvoorwaardenbeleid personeelsbeleid","persoonsverheerlijking","pesten","peuterspeelzalen","piano's pianofortes","pijn","pingu\u00efns","piramiden","pizzeria's","plafonds","plaggenhutten","plagiaat","planetaria","planeten","planten","plantenziekten","plastische chirurgie","gastgezinnen pleeggezinnen pleegouders","pleegkinderen","plunderingen","plutonium","poema's","epigrammen gedicht gedichten klankpo\u00ebzie po\u00ebzie verbosonie\u00ebn","pokken","polemologie vredespolitiek vredeswetenschappen","kinderverlamming polio","agenten politieagenten","politieke gevangenen","politiescholen","fierljeppen polsstokverspringen","polygamie","pony's","poorten","popart","beatmuziek popmuziek","poppenkast poppentheater schimmenspel wajang","pornografie","postduiven","postkantoren","pottenbakkerijen pottenbakkers","potvissen","premi\u00e8res","prijscompensatie","prikklokken","primitieve landbouw","privacy","privatisering","projectontwikkelaars projectontwikkelingsmaatschappijen","prostitutie","protectionisme","prothesen","provinciehuizen","provincies","provo's","pruiken","psychiatrische ziekenhuizen","psychoanalyse","gedragstherapie psychotherapie","puberteit","publiek toeschouwers","puzzels","rabbijnen","racisme rassendiscriminatie","radioactiviteit stralingsgevaar","radiostations radiozenders","radiotelescopen","radiotoestellen","rages","rampenplannen","ratten","rattenvangers","raven","ravijnen","realisme","kritieken recensies","ontvangsten recepties","recherche","arrondissementsrechtbanken kantongerechten rechtbanken","rechtsbijstand","kinderrechtswinkels rechtswinkels wetswinkels","recitals","reclame","reclamebureaus","reclassering","records","recycling","redacteuren","reddingen","reddingsbrigades","rederijen","ree\u00ebn","reformwinkels","regenbogen","regenwouden tropische regenwouden","cineasten filmregisseurs hoorspelregisseurs radioregisseurs regisseurs televisieregisseurs theaterregisseurs toneelregisseurs","reigers","re\u00efncarnatie wedergeboorte","reisbureaus","reisgidsen","reisleiders","rekenen","rekenmachines","rekruten","rekwisieten","relatiegeschenken","relaties","gewijde muziek kerkmuziek religieuze muziek","monstransen religieuze voorwerpen","relikwie\u00ebn","rellen","renaissance","renbanen","rendieren","renovaties","bedrijfssaneringen reorganisaties saneringen","requiems","reservisten","resoluties","restauraties","artritis artrose reuma ziekte van Bechterew","re\u00fcnies","revalidatie","communes samenlevingsvormen woongroepen","concubinaat samenwonen","sancties","savannen","Saxofoons saxofoons s\u00e1xofoons","scenario's","schaken","schaatsen (voorwerpen)","schadevergoedingen smartengeld","schansspringen skispringen","scharrelkippen scharrelvee","grensrechters scheidsrechters","schietbanen","schietpartijen","kleiduivenschieten schietsport","schilders","schildpadden","schillenboeren","schimmels","binnenschippers scheepvaartpersoneel schippers","schizofrenie","schoeisel","schoenenindustrie schoenfabrieken","schoenmakers","schoenpoetsers","scholeksters","schoolkeuze","schoolverlaters","schoolverzuim spijbelen","schoonheidsspecialistes schoonheidsverzorging","missverkiezingen schoonheidswedstrijden","schoonmaakbedrijven","schoonspringen","schoorsteenvegers","schorpioenen","schorren","schotelantennes","schrijfvaardigheid schrijven","schroot","schuilkelders","schuilkerken","schulden","scooters","seismologie","seizoenarbeid","seksuologie","shinto\u00efsme","showballet","Siamese tweelingen","sieraden","sigarenmakers","silo's","simulaties simulators vluchtsimulators","sisal","sjoelen","skeletten","skinheads","slaapmiddelen","narcolepsie slaapstoornissen","slachten","slagerijen slagers","slakken","slangen","slangenbezweerders","slapen","slapsticks","slavernij","slechtzienden","sleepvaart","slib","sloepen","sloperijen","sluikreclame","sluizen","smaakstoffen","amateurfilms amateuropnamen home movies home videos priv\u00e9films smalfilms","smeden","smokkelen","cafetaria's snackbars","snacks","snelrecht","snoepgoed","snooker","sociale voorzieningen sociale zekerheid toeslagen verzorgingsstaat","sociologie","bestuursoverdracht soevereiniteitsoverdracht","apps (computerapplicaties) computerapplicaties software","soja","solliciteren","songfestivals","soul","souvenirs","sparen","specerijen","spechten","buitengewoon onderwijs lom-scholen speciaal onderwijs zmlk-scholen","speculatie","speedboten","speedway","speelhallen","speelplaatsen speeltuinen","spelen","sperma","spieren","spijkerschrift","spinnen","spionage","spiritisme","spoken","sponsoring","sportaccommodaties","sportartikelen","blessures sportblessures","sportleraren","spots","spraakorganen","spreekkoren yells","gezegden spreekwoorden","spreeuwen","sprinkhanen","sprookjes","squash","staal","staalfabrieken staalindustrie","staatsgrepen","stadions","deelgemeenteraden stadsdeelraden","stadspoorten","sociale vernieuwing stadsvernieuwing","stamhoofden","stammen","normalisatie standaardisatie","statelozen","stations","statistiek","stedenbouw","steelbands","steenbokken","kalksteengroeven mergelgroeven steengroeven","stekelvarkens","stem","pampa's prairies steppen toendra's","sternen","melkweg sterren","sterrenbeelden","observatoria sterrenwachten","stichtingen","stierengevechten","ossen stieren","stijldansen","stilte","stofwisselingsziekten ziekte van Fabry","stomerijen","stoommachines stoomwalsen","stormvloedkeringen","straatverlichting","straatvoetbal","stranden","strandjutters","stratenmakers","streekvervoer","stress","stropen","struisvogels","studenten","studentenverenigingen","studiefinanciering","stukgoederen","stuntlieden","stuntrijden stunts stuntvliegen","subsidies","suikerbieten","diabetes suikerziekte","supporters","surrealisme","symfonie\u00ebn","synagogen","downsyndroom syndroom van Down","cystic fibrosis taaislijmziekte","taalstrijd","tabak","tachografen","taekwondo","tafeltennis","tafelvoetbal","tai chi","talentenjachten","tandems","orthodontie tandheelkunde","tango","tao\u00efsme","tapijtweverijen","taxateurs","taxichauffeurs","tbs terbeschikkingstelling","ambachtsscholen hts lts mts technisch onderwijs","technische hogescholen technische universiteiten","techniek technologie","dierenwelzijn","cartoonisten sneltekenaars technisch tekenaars tekenaars","kindertelefoon telefonische hulpdiensten","telefonisten","telefooncellen","telefooncentrales","telescopen","televisiestations televisiezenders","televisietoestellen","pagodes tempels","tenten","termieten","terpen wierden","testamenten","tests","textiel","katoenindustrie lakenindustrie textielindustrie","theeplukkers","theologie","gnosticisme theosofie","therapie\u00ebn","telewerken thuiswerk","tieners","periodieken tijdschriften","tijgers","timmermannen","tin","sportwedstrijden toernooien","tolheffing","tomaten","lekespel toneel","toneelscholen","tonijnen","touwfabrieken touwslagerijen","tractoren","trailers","trainers","transplantaties","expeditiebedrijven transportbedrijven","transgender transgenderisme","KZ-syndroom oorlogssyndromen trauma's","travestie","triatlon","tribunalen","tribunes","trolleybussen","abdicaties troonopvolging","tropische planten","tbc tuberculose","tuinarchitectuur","tuinders","tuinen volkstuinen","tulpen","tumoren","turnen","tweelingen","twist","tyfus","ufo's vliegende schotels","uilen","uiterwaarden","avondklok uitgaansverbod","uitgeverijen","uitkijktorens","hondenpoep ontlasting uitwerpselen","uitwijzingen","uitzendbureaus","urologie","utopisme","vaarbewijzen","inentingen injecties vaccinaties vaccins","discuswerpen","hink-stap-springen","kogelstoten","meerkamp tienkamp vijfkamp zevenkamp","sprint","skeleton","noordse combinatie","speerwerpen","skateboarden","design vormgeving","horecapersoneel hotelpersoneel kelners portiers","marktkooplui standwerkers","krantenverkopers straatverkopers","duikers kikvorsmannen","longen","nieren","darmen","lever","hart","infolijnen telefonische informatielijnen","commerci\u00eble telefoonlijnen sekslijnen","kliklijnen tiplijnen","bedrijfsfilms","religieuze programma's","televisiedrama's","blijspelen komedies","comedy's","consumentenprogramma's","culinaire programma's kookprogramma's","culturele programma's kunstprogramma's","milieuprogramma's natuur- en milieuprogramma's","educatieve programma's schoolradio schooltelevisie","erotische programma's","bioscoopjournaals journaals","journaals nieuwsbulletins nieuwsuitzendingen","filmprogramma's","vakantiebeurzen","vakantiehuizen zomerhuizen","jeugdkampen vakantiekampen","valken","valkeniers","valsemunterij","valuta","vari\u00e9t\u00e9 vlooiencircus","paden","jamboree padvinderij scouting","pakhuizen","paleizen","natuurgebieden stiltegebieden","natuurrampen","leisteen marmer natuursteen","kloosterlingen kloosterzusters nonnen","noodtoestand","jungle oerbossen oerwouden","oorlogsschade","oorlogsvrijwilligers","openbaar onderwijs","openbare voorzieningen","openingen","opera's","operaties","oorlogsmisdaden oorlogsmisdadigers","ongedierte","ongelukken ongevallen","ontdekkingsreizen","ontsmettingen","audi\u00ebnties ontvangsten","onweer","oorlogen","erebegraafplaatsen oorlogsgraven","piloten","razzia's","rechters","kort geding processen","recreatiebedrijven","reddingsboten","calamiteiten rampen","radar","exposities tentoonstellingen","wapens","olieraffinaderijen raffinaderijen","reparaties","repatri\u00ebring","reptielen","republiek","residenties","kantines restaurants","reinigen schoonmaken wassen","passagiers reizigers","pluimvee","poliklinieken","politici","politicologie politieke wetenschappen","politionele acties","pompen","pontonbruggen","pleziervaart","plantages","plattegronden","platteland","ceremonies plechtigheden","pleinen","poppen","prinsen","prinsessen","granaten projectielen torpedo's","propaganda","psychiatrie","psychologie","portret portretten","postbodes","prauwen","dominees predikanten","prenten","paters priesters","prijsuitreikingen","nationalisatie","landmacht","landwegen","leeftijdsgroepen","leerlingen scholieren","leerplicht parti\u00eble leerplicht","lente","kwekerijen","laboratoria","ladders","lampen","lanceerbases","lanceringen","landaanwinning landwinning","landbouw","landbouwwerktuigen ploegen","landhuizen","leraren","loodsen loodswezen","loopgraven","loterijen","luchtmacht","luchtopnamen","luchtvaart vliegen (luchtvaart)","maaltijden","locomotieven","colleges lessen","lichaamsverzorging","lichamelijk gehandicapten","lichamelijke opvoeding","liften","lijfstraffen","doden lijken","lijkverbrandingen","letterkunde literatuur","kustvaart","abdijen kloosters","berlines koetsen","koffers","koffie","gouvernement koloniaal bestuur","koloni\u00ebn overzeese gebiedsdelen","koningen","klokken uurwerken","kiesrecht stemrecht","kinderarbeid","cr\u00e8ches kinderdagverblijven","kinderen","klederdrachten volksdrachten","badkleding kleding","kleinkinderen","kleuters peuters","vasten","veehouderijen","veemarkten","veenkoloni\u00ebn","veevoeder","veganisme vegetarisme","veilingen","veldlopen","veldrijden","vendelzwaaien","vennen","verblijfsvergunningen visa","candid camera verborgen camera","verbouwingen","drugs narcotica verdovende middelen","verf","overleggen vergaderingen","gif vergiftigingen","vergrijzing","ondervragingen verhoren","verhuizingen","verkeersbeleid","verkeersborden","verkeersonderwijs","verkeerstorens","verlegenheid","verliefdheid","verloskunde","verlovingen","afslanken vermageren","verdwijningen vermissingen vermiste personen","verpakkingsindustrie","inpakmachines verpakkingsmachines","verpleeghuizen","urbanisatie verstedelijking","tolken vertalers","vertegenwoordigers","falsificaten vervalsingen","ruilbeurzen verzamelbeurzen","verzamelingen","assuranti\u00ebn verzekeringen","verzekeringsmaatschappijen","verzilting","verzuiling zuilen","verzuring zure regen","dierenziekten veterinaire ziekten","veulens","video-apparatuur videorecorders","videocassettes","videoclips","videokunst","videotheken","vijvers","villa's","groenlingen putters vinken","altviolen violen","virtual reality virtuele realiteit virtuele werkelijkheid","virussen","visafslag","visindustrie visverwerkingsindustrie","visserijquota","vitamines","vleermuizen","vleesindustrie","vleeswaren","pilotenopleidingen vlieglessen vliegopleidingen","vliegtuigindustrie","vlinders","vloerbedekking","vlooien","vlooienmarkten","vlootoefeningen","vlotten","politieke vluchtelingen vluchtelingen","tentenkampen vluchtelingenkampen","vluchtstroken","voedingsleer","voetbal","voettochten","vogelspinnen","ornithologen vogelkenners","volksspelen","volksuniversiteiten","legendes sagen volksverhalen","open school volwassenenonderwijs vormingswerk","vondelingen","vonnissen","voodoo","voogdij","anticonceptie voorbehoedmiddelen","voortplanting","vossen","vredesbeweging","vluchtelingenbeleid vreemdelingenbeleid","vreemdelingenpolitie","vriendschap","vrijmetselarij","vrouwenbeweging","vrouwenemancipatie","vrouwenhandel","genderstudies vrouwenstudies","vruchtensappen","vuilnisbelten vuilstortplaatsen","VUT vervroegde uittreding","vuur","vuurvreters","geweren vuurwapens","vuurwerk","voorbereidend wetenschappelijk onderwijs vwo","steltlopers waadvogels","waaggebouwen","wadlopen","walrussen","walvissen","walvisvaarders","walvisvangst","wandelen wandelsport","gobelins wandtapijten","wapenembargo's","wapenindustrie","wapenstilstand","warenhuizen","wasberen","wasmiddelen","wassen beelden","wasserijen","waterbuffels","waterfietsen","waterhoentjes","waterleidingbedrijven","waterlelies","waterplanten","waterpolitie","waterpolo","hoogheemraadschappen waterschappen","watervogels zeevogels","waterzuivering","wederopbouw","weduwen","weduwnaars","weekdieren","weersverwachtingen","weeshuizen","wegenwachters","weilanden","welvaart","club- en buurthuiswerk welzijnswerk","wereldrecords","wereldreizen","wereldtentoonstellingen","wereldwinkels","arbeidsmarkt werkgelegenheid","werkgeversorganisaties","werkloosheid","werkplaatsen","werkvergunningen","tornado's wervelstormen windhozen","wespen","westerns","wetsontwerpen","weverijen","marters wezels","wezen","whiplash","wiegendood","wijkverpleging","wijn","druivenpersen wijnbouw","wijnkelders","wilde zwijnen","wildwatervaren","knotwilgen wilgen","plankzeilen windsurfen","winkelcentra","winstdelingen","winti","wisenten","wiskunde","wisselkantoren","woestijnen","wol","wolken","wolkenkrabbers","wolven","wonderen","huizenbouw woningbouw","woningbouwverenigingen woningcorporaties","interieurs woninginterieurs","woningnood","woonboten","woonwagens","stadswijken woonwijken","dyslexie woordblindheid","woordenboeken","pieren wormen","wulpen","yoga","zaalsporten","zaligverklaringen","zand","zandopspuitingen","zandverstuivingen","zandwinningen","aubades serenade zanghulde","zangvogels","zebra's","oceanen zee\u00ebn","zeefdrukken","zeehonden","zeekoeien","zeeleeuwen","zeeolifanten","zeep","piraten zeerovers","zeevaartscholen","zeezeilen","zelfkastijdingen","zelfverbranding","zelfverdediging","zenboeddhisme","zendamateurs","televisietorens zendmasten","zenuwgassen","zenuwstelsel","ziekenfondsen","ziekte van Parkinson","ziekteverzuim","zigeunermuziek","zigeuners","zijde","zink","zionisme","zolders","zomertijd","zondagsrust","zonnebloemen","zonnepanelen","zonnestelsels","eclipsen zonsverduisteringen","zuivelindustrie","zuurstof","zwaartekracht","zwaluwen","zwanen","zwangerschappen","zweefvliegen","straatkinderen zwerfkinderen","accountants","alternatieve geneeswijzen haptonomie iatrosofie iriscopie natuurgeneeskunde","aquacultuur viskwekerijen","amateursport","aantredingen ambtsaanvaardingen benoemingen be\u00ebdigingen eedafleggingen inauguraties installaties","arbodiensten","antiquariaten","arbitrage conflictregeling","arrangeurs","auteursrecht copyright","banken bankwezen","basisinkomens","basisvorming middenschool","bedrijfsbezettingen bezettingen","bestuurlijke herindeling gemeentelijke herindeling","boekverbrandingen","cajun","Collectieve arbeidsovereenkomst cao's collectieve arbeidsovereenkomsten","casting","cd's compact discs","choreografen","columnisten","columns","consumptie","criminalistiek criminologie","cruises","doop dopen","draf- en rensport","essays","esthetica schoonheidsleer","existentialisme","aerobics fitness","fosfaten","freelancers zelfstandigen zonder personeel zzp'ers","geestelijke liederen gezangen psalmen religieuze liederen","besmettelijke ziekten overdraagbare ziekten","echo's geluid","geluidskunst","goochelaars goochelen illusionisten","grondstoffen","dotteren hartchirurgie","historische romans","huurbeleid","idolen","geheime diensten inlichtingendiensten","insecten","jeugdliteratuur kinderboeken kinderliteratuur kinderverhalen","kluizenaars","kunstenaarskolonies","kustverdediging","Identiteitskaart Identiteitskaarten legitimatiebewijzen persoonsbewijzen","liturgie","inkomensbeleid loon- en prijsbeleid","maatschappijleer","magisch realisme","makelaars","marktonderzoek","militarisme","musicologie muziekwetenschap","muziekonderwijs muziekscholen","neutraliteit","new age","nivellering","overlijden sterven","ozonlaag","parkeerbeleid","pausen","piratenzenders radiopiraten televisiepiraten","pluimveehouderijen","polarisatie","producenten producer producers","muziekproducer muziekproducers producers","productschappen","protestliederen","public relations","referenda volksraadplegingen","regionale omroepen","reinigingsdiensten stadsreiniging straatvegers vuilnismannen","bemiddelingsbureaus relatiebemiddeling","retorica spreekvaardigheid welsprekendheid","sacramenten","salsa","segregatie","ongewenste intimiteiten seksuele intimidatie","shorttrack","skaten skeeleren","sociale werkplaatsen sociale werkvoorziening","sociale wetenschappen","softbal","spaarregelingen","stenografie","straattheater","streekromans","taalgebruik","tabaksindustrie","toekomstvoorspellingen waarzeggen","transfers","damesromans triviaalliteratuur","herkenningsmelodie\u00ebn tunes","vertelkunst","voedingsmiddelenindustrie","in-vitrofertilisatie ivf kunstmatige inseminatie reageerbuisbevruchting voortplantingstechnologie","akkoorden verdragen vredesakkoorden","recreatie vrijetijdsbesteding","welzijn","zangonderwijs","harakiri zelfdoding zelfmoord","zenderindeling zenderkleuring","ziekenomroepen","workshops","masterclasses","repetities","praatradio talk radio","wetenschapsfilosofie","baanwerkers spoorwegwerkers","houseparty's","strijdliederen","aardbeien","natuurbeheer natuurbescherming","huisraad","sjiieten","liedjesschrijvers tekstschrijvers","hardrock","hoorspelen","aanrandingen ontucht seksueel geweld seksuele delicten zedendelicten zedenmisdrijven","Mariaverering","migrantenomroepen","raad van commissarissen raden van commissarissen","bodyguards lijfwachten","audiovisuele archieven filmarchieven geluidsarchieven","soaps","publieke omroep","esoterie spiritualiteit","begraafplaatsen dierenbegraafplaatsen erebegraafplaatsen grafheuvels kerkhoven","plagen","arbeidsvoorwaarden","herkenningsmelodie\u00ebn jingles","impresariaat","beachvolleybal","kroongetuigen","eeuwwisselingen millenniumwisselingen","nutsbedrijven","herenigingen","achtbanen attractieparken pretparken","computercriminaliteit computerinbraak cyberaanvallen cybercrime phishing","kartelvorming","wereldmuziek","doofstommen spraakstoornissen stotteren","levensliederen schlagers smartlappen","belagingen stalking","eiken","paardensport","vrijwilligers vrijwilligerswerk","straffen","gesneuvelden oorlogsinvaliden oorlogsslachtoffers","scholen schoolgebouwen","modelbouw modelschepen modeltreinen modelvliegtuigen","munt munten","musicals","instrumentalisten musici muzikanten","muurkranten","fresco's muurschilderingen wandschilderingen","universiteiten","milieubeleid","nationaalsocialisme nazisme","nationale feestdagen","nationale parken","mummies","moerassen","moesson","molens windmolens","koningskwesties koninkrijk monarchie vorstendom","kloosterlingen monniken","monumenten","moskee\u00ebn","motels","motoren","motorfietsen","wielrennen","mariniers","satellieten","schilderen","markten","marsen","mascottes","matrozen","medailles prijzen (medailles)","musea","machines werktuigen","mannen","management","maneges","marechaussees","marine","meerlingen","legerplaatsen militaire bases","landingen luchtlandingen militaire operaties","militaire vliegtuigen","militaire vliegvelden vliegbases","legervoertuigen militaire voertuigen","militairen soldaten","trams","mijnbouw","barometers decibelmeters geigertellers meet- en weeginstrumenten thermometers thermostaten weeginstrumenten zandlopers","meiden meisjes","melk","meren","metalen","metingen","metro's ondergrondse","metselen","meubilair","riet","vissershavens","vissersschepen","dierproeven proefdieren vivisectie","vlees","verkeersvliegtuigen vliegtuigen","verzetsstrijders","bejaardentehuizen verzorgingstehuizen","forten vestingen","viaducten","video","traditionele dansen volksdansen","volkskunst","volksliederen","volksmuziek","volleybal","overheidsvoorlichting voorlichting","vorsten vorstinnen","folklore tradities volkscultuur","vlootschouwen","levensmiddelen voedingsmiddelen","koken recepten voedselbereiding","voedseldroppings voedselhulp","voetgangers","trottoirs voetpaden","vogels","verzet","vandalisme vernielingen","varkens","vee","veen","veeteelt","tampons verbandmiddelen","benzinemotoren verbrandingsmotoren","vaders","turf","uitvindingen","uranium","verdedigingswerken","verlof","verpleegkundigen ziekenverzorgers","barricaden versperringen","vertalingen","vervoermiddelen voertuigen","verwarming","kachels verwarmingsapparaten","vernielingen verwoestingen","verjaardagen","stoplichten verkeerslichten","koningshuis koningshuizen vorstenhuizen","worstelen","autowrakken scheepswrakken vliegtuigwrakken wrakken","zebrapaden","scheepvaartpersoneel zeelieden","sk\u00fbtsjes zeilschepen","zeilen","zenders","wind","windenergie","winkels","winter","wintersport","woonwagenkampen","zuivelproducten","klimaat","krijgers","krotten","kruisers","kunst","beeldend kunstenaars kunstenaars","kunstgeschiedenis","kunststoffen","kusten","kransleggingen","koopvaardij zeescheepvaart zeevaart","koopvaardijschepen","terrorisme","koper","maatschappelijk werk","onderzoek","regen","mist","ME (mobiele eenheid) mobiele eenheid","mobilisatie","sportwedstrijden wedstrijden","verkeer","mishandeling","kerkklokken torenklokken","torens","mammoettankers olietankers tankers tankschepen","standbeelden","steen","stoomschepen","stoomtrams","stormen windhozen","staatshoofden","staatkunde staatsinrichting","staatsrecht","staatsvormen","gemeentehuizen stadhuizen","rietsuiker suiker","suikerriet","sultans","supermarkten","symbolen","taalonderwijs","tandartsen","stuwmeren","straatmeubilair","straten","krijgsmacht leger strijdkrachten","cartoons strips stripverhalen","studio's","stuwdammen","stuwen","hangjongeren","amfibievoertuigen","animaties computeranimaties","animisme natuurgodsdiensten","annexaties inlijvingen","ansichtkaarten kaarten prentbriefkaarten","antennes","apparatuur","aquaducten","ambachten nijverheid","adoptie","advertenties","afbraak sloop slopen","afgevaardigden delegaties","agressie","aardwarmte alternatieve energie duurzame energie getijdenenergie groene energie hernieuwbare energie","aluminium","atletiek","auteurs schrijvers","auto auto's","autobussen bussen (autobussen)","automaten","ateliers","arbeidsrecht","blokkades","boeddhisme lama\u00efsme","boeken","boerderijen","boeren","boetedoening","boksen","bomen","basketbal","batik","bauxiet","bedelaars","beeldende kunsten","beeldhouwkunst","begrafenissen","basilieken","badplaatsen","bagage","baggerwerken","balkons","Papiergeld bankbiljetten","barakken","bergen gebergte","alpinisme bergsport klim- en bergsport","ambten beroepen","beroepsonderwijs vakopleidingen","bestrating plaveisel","beken","belangenorganisaties","belegeringen","benzine","benzinepompen benzinestations tankstations","bezoeken ontvangsten","onderhandelingen","ministers-presidenten premiers","jeugd jongeren","sociale uitkeringen uitkeringen","ambtenaren","doema's parlementen senaten volkscongressen","levensstandaard welvaartsindex","belastingen fiscus","stakingen werkonderbrekingen","waterverontreiniging","tuinbouw","bedrijf bedrijfsleven bedrijven ondernemingen","vrouwen","ontwikkelingshulp ontwikkelingssamenwerking","enqu\u00eates opiniepeilingen","presbyterianisme protestantisme","wetgeving wetten","internationale organisaties","jeugdorganisaties","pers","troonredes","geestelijken geestelijkheid","kerkgebouwen","missie","geld","banenplannen werkgelegenheidsbeleid","omroep","educatie onderwijs","sociaal beleid sociale vernieuwing","Algemene verkiezingen gemeenteraadsverkiezingen presidentsverkiezingen verkiezingen","kabinetten ministerraden regering regeringen","jodendom joodse religie sabbat","openbaar vervoer","christendemocratie confessionalisme","politieke stromingen","anarchisme","bolsjewisme communisme","politiek","liberalisme","socialisme","marxisme","arbeiders","christendom","posterijen postwezen","brieven","bandrecorders cassetterecorders cd-spelers datrecorders geluidsapparatuur koptelefoons luidsprekers megafoons mp3-spelers","Jodenvervolging pogroms","grammofoonplaten","ziekten","astmaklinieken herstellingsoorden kuuroorden revalidatiecentra sanatoria","gijzelingen","zweefvliegtuigen","zwembaden","zwemmen","zoutwinning","ziekenvervoer","zilver","zomer","zonne-energie","zoogdieren","strooizout zout","wapenschilden","warmte","waterkracht","waterkrachtcentrales","containerschepen vrachtschepen","vrouwenarbeid","vulkanen","watermolens","wegen","werkgevers","werktuigbouwkunde","wetenschap","tertiair onderwijs universitair onderwijs wetenschappelijk onderwijs","wetenschappelijk onderzoek","waterski\u00ebn","watervallen","watervliegtuigen","waterwegen","waterbouw waterbouwkunde waterwerken weg- en waterbouw","weer","tsaren","tropen","sigaren","sigaretten","tunnels","abattoirs slachterijen","slachtoffers","arresleden slee\u00ebn","sleepboten","sloten","sportverenigingen","schuren","morseseinen seinen","seizoenen","sekten","sjeiks","taal","strafrecht","sluipschutters","spoorwegovergangen","sportmanifestaties sportwedstrijden","sportvliegtuigen ultralights","cartoons karikaturen karikatuur spotprent spotprenten","explosieven kruit springstoffen","drinkwatervoorziening watervoorziening","sneeuw","autosnelwegen snelwegen","meerkoet meerkoeten","flipperkasten gokautomaten speelautomaten","speelfilms","speelgoed","schrijfwaren","rondvaartboten","rookwaren","rouw","rubber","ruimtevaart","ru\u00efnes","rijst","rijtuigen","ceremonies rituelen","delta's rivieren","roeiboten","runderen","scheepsbouwkunde","scheepvaart","giften schenkingen","boten schepen","schilderijen","schilderkunst","schoorstenen","schrijfmachines typemachines","jachtwerven scheepsbouw scheepswerven","rupsvoertuigen","safari's","sanitaire voorzieningen toiletten","maquettes modellen schaalmodellen","schapen","multiculturele samenleving","staatsbezoeken","tennis","terrassen","muziektheater","dopen tewaterlatingen","theater","schouwburgen theaters","thee","tanks","tapijten","tatoeages","taxi's","technische wetenschappen","tehuizen","tekenkunst","telecommunicatie","radiotelegrafie telegrafie","touw","trappen","trekschuiten","timmeren","toerisme","tolwegen","geweld zinloos geweld","politie","sport","personeel werknemers","immigratie","overheidsbeleid regeringsbeleid","jeugdbeleid","bewindslieden ministers","departementen ministeries","jubilea","kernreactors reactorvaten","petities verzoekschriften","gemeenten","fabrieken","homoseksualiteit","kolen steenkool","elektriciteitscentrales","voorzitters","bijzonder onderwijs levensbeschouwelijk onderwijs","muziek","milieu","recht rechtswetenschappen","atoomenergie kernenergie reactorvaten","burgerrechten grondrechten mensenrechten rechten van de mens","artiesten kunstenaars","huren huur pachten","aandelen effecten obligaties opties waardepapieren","kerken","bossen","onderscheidingen oorkonden prijzen (onderscheidingen)","radio","democratie","onafhankelijkheid onafhankelijkheidsverklaringen","bevolking demografie","moeders","rechtspraak","treinen","televisie","mineralen voeding voedsel","industrie","energiebeleid","persconferenties","aannemers bouwbedrijven bouwnijverheid","vakbeweging vakbonden","publicaties","dagbladen kranten vrouwenkranten zondagskranten","humanitaire hulp medische hulp vluchtelingenhulp","gedenkdagen herdenkingen","fascisme","feminisme","zang","comit\u00e9s commissies","gemeenteraden","steden","seks seksualiteit","conferenties congressen forums symposia","invasies","buitenlands beleid","bijeenkomsten","kersttoespraken nieuwjaarstoespraken redevoeringen toespraken","kampen tentenkampen","vrede","aardgas","gemeentebesturen","wethouders","mijnen","hospitalen klinieken ziekenhuizen","hulpverlening","munitie","transport vervoer wegtransport","luchthavens vliegvelden","huizen woningen","acties protesten","omroeporganisaties zendgemachtigden","journalisten","inkomens","havens","arbeid werk","kabinetscrises ministerscrises regeringscrises","atoombommen atoomwapens kernwapens neutronenbommen","onderwijsgevenden","brandstoffen fossiele brandstoffen stookolie","grensovergangen","groenten","groenteteelt","grondbewerking","grondwetten","grootouders","grotten","gravures","graanproducten","grachten","graffiti","grafiek grafische kunsten typografie","grammofoons platenspelers","granen tarwe","grafstenen graftombes graven","familiewapens heraldiek stadswapens","herfst","heuvels","hijskranen kranen","hindoe\u00efsme","hobby's","hockey","hofhoudingen","hofjes","helikopters","gymnastiek","handenarbeid","handtekeningen","handwerken","quarantaine","hangars","heesters struiken","heide","betalingsverkeer geldverkeer","gemalen stoomgemalen","geneeskunde","geneesmiddelen injecties kalmeringsmiddelen medicijnen pillen vaccins","genie","genocide volkerenmoord","cadeaus geschenken","geschiedenis historie","geheugen","gebak taarten","bevallingen geboorten","gebouwen","gedenkstenen gedenktekens","gedrag","geestelijk gehandicapten verstandelijk gehandicapten","geestelijke gezondheidszorg","gehandicapten invaliden mensen met een beperking oorlogsinvaliden","geschut kanonnen mortieren","bordspelen gezelschapsspelen spelletjes","gezondheid volksgezondheid","glas","godsdienstoefeningen","goederenvervoer vrachtvervoer","golfbrekers pieren","goud","autoriteit gezag macht","geslacht sekse","gedetineerden gevangenen","gevangenissen","gevechten","gevels","landbouwgewassen","gewonden","juristen","juwelen kroonjuwelen","kaartspelen","kaas","kabelbanen skiliften","kampementen legerplaatsen","kamperen","kanalen vaarten","islam mohammedanisme soefisme","jachthavens","jachtvliegtuigen straaljagers","jeeps","jeugdcriminaliteit","jeugdherbergen","jongens","transportvliegtuigen vrachtvliegtuigen","varkenspest","Kamerdebatten parlementaire debatten","verkiezingsdebatten","partijprogramma's verkiezingsprogramma's","vliegdekschepen","wapenhandel wapenleveranties","longkanker","zendelingen","zending","waterkanonnen","estafettes","jeugdprogramma's kinderprogramma's","jeugdprogramma's jongerenprogramma's","medische programma's","misdaadprogramma's","muziekprogramma's","oral history","politieke programma's","bumpers promo's","propagandafilms","belspelletjes spelprogramma's","quizzen","hersenvliesontsteking nekkramp","honderdjarigen hoogbejaarden","stervensbegeleiding","kadavers","sk\u00fbtsjesilen","medische keuringen","medische fouten","menselijk leed","voeten","radioactieve stoffen","ogen","uitbuiting","stralingsziekten","verloskundigen vroedvrouwen","botanie plantkunde","journalistiek","kazernes legerplaatsen","keizers","kermissen","kerosine","keukens","kathedralen","gokken kansspelen piramidespelen","kantoren","karren","kassen","kastelen","inundaties","bestek gebruiksvoorwerpen huishoudelijke artikelen serviezen","huishouding","huiskamers","consigne statiegeld","huisvesting volkshuisvesting wonen","huisvrouwen","huldigingen","hunebedden","hutten","huwelijken trouwen","hygi\u00ebne","butlers dienstmeisjes huishoudelijk personeel werksters","stilleven stillevens","honger hongersnood","honkbal","hoogleraren professoren","hooi","clubs horeca","horloges uurwerken","hotels","hout","houtbewerkers","huisdieren","'inheemse' bevolking natuurvolken","inpolderingen","akkoorden internationale akkoorden internationale verdragen verdragen","internationaal recht volkenrecht","betrekkingen internationale betrekkingen internationale samenwerking","interneringskampen jappenkampen","interviews ondervragingen vraaggesprekken","intochten","'inheems' bestuur","ijs","ijsbanen kunstijsbanen","bandy ijshockey","ijzer","ijzererts","feestverlichting illuminatie","illustraties","industri\u00eble vormgeving","infanterie","akkoorden contracten overeenkomsten","controles inspecties","crematies","criminaliteit delicten misdaad misdrijven","conflicten ruzies","componisten","computers","conci\u00ebrges huisbewaarders","delfstoffen ertsen gesteenten mineralen","deportaties","detailhandel","diamanten","alleenheerschappij dictatuur junta","diefstal","defil\u00e9s","betrekkingen culturele betrekkingen culturele uitwisselingen","cultures","curiosa","dammen","damsport","jacht","burgers","burgerwacht","cabaret kleinkunst","bars caf\u00e9s pubs","caissons","camera's televisiecamera's","camouflage","campings","burgemeesters","brandweer brandweerlieden brandweermannen brandweervrouwen","bromfietsen","brood","bruggen","bruinkool kolen","bungalows","circussen","beschaving civilisatie","collecties","reizen","veiligheid","vrijheid van meningsuiting","persvrijheid","dieren","cavalerie huzaren","censuur","chemische industrie","dierentuinen","excursies rondleidingen","expedities","faillissementen","families","federalisme","feestdagen","feesten party's","energiebronnen","energieverbruik","elektriciteitsvoorziening energievoorziening","epidemie\u00ebn","eredoctoraten","crossfietsen fietsen mountainbikes rijwielen snorfietsen","fotografie","fregatten","fruit vruchten","fruitteelt","gamelan","garages","gas","onderwijsbeleid","fotografen","fietspaden","files","cinema cinematografie film","flatgebouwen","flessen","fonteinen","foto's","dokken droogdokken","doodstraf kruisigingen","dorpen","doofstommen doven","dranken","droogte watergebrek","droppings","dodenherdenkingen","diesel dieselolie gasolie","dijken","ambassadeurs consuls diplomaten","dirigenten orkestleiders","dochters","aktes documenten dossiers","energie","elektriciteit elektrische energie","elektrische apparatuur","elektromotoren","elektronica","bruingoed elektronische apparatuur","emballage verpakkingsmaterialen","EHBO eerste hulp bij ongelukken","duinen","echtscheidingen scheidingen","edelstenen","longaandoeningen longemfyseem stoflongen","pedagogiek","presentatoren programmamakers","novellen proza romans","verkrachtingen","voordrachtskunst","anatomie ontleedkunde","zintuigen","DNA chromosomen genen","fysica natuurkunde","lijsttrekkers","fjorden","ijsbergen","goede doelen liefdadigheid","nationalisme","optische instrumenten","valleien","alcoholische dranken","audiovisuele media","bouwmaterialen","co\u00f6peraties","genotsmiddelen","geologie","kaarsen","lingerie","mythologie","pantservoertuigen pantserwagens","polo","revues","cartoons tekenfilms","exacte wetenschappen natuurwetenschappen","aftredingen afzettingen impeachment","arbeidersbeweging","onthullingen","dancings discotheken","anatomie (dieren) dierkunde zo\u00f6logie zo\u00f6tomie","knipsels krantenknipsels","bio-industrie kistkalveren legbatterijen","keramiek","aardewerk","antimilitarisme pacifisme","concilies kerkvergaderingen pastorale concilies synodes","octrooien patenten","jeugdculturen subculturen","laptops pc's personal computers tablet-pc's","ecg elektrocardiogram","beroepsverboden","faxapparatuur plotters printers randapparatuur scanners","bronnen waterbronnen","edelmetalen","buitenaards leven","chemische elementen stikstof","ambulances ziekenauto's","gevangeniswezen penitentiaire inrichtingen strafinrichtingen","roeien","beeld- en geluidsdragers beeldplaten cd-video geluidsdragers videoplaten","geloof gelovigen godsdiensten religies","accordeons harmonica's trekharmonica's","jogging trimmen","vocalisten zangeressen zangers","voedseltekort voedseltekorten voedselvoorziening wereldvoedselsituatie","schermen","ski\u00ebn","stiefgezinnen stiefouders","kaarten landkaarten","cultuurfilosofie","uitkeringsgerechtigden","alligators kaaimannen krokodillen","geldsanering monetair beleid","zaden","guerrilla's opstandelingen rebellen","ge\u00efmproviseerde muziek","satire","vlaggen","vaandels","vakantie","ponten veerboten","verlichting","verslavingen","verstekelingen","visserij","wielersport","telefonie","schaatsen","luxeartikelen luxegoederen","politieke partijen","zelfbediening","geloof levensbeschouwingen","grenscorrecties grenzen","rijbewijzen","uniformen","filmgenres","stoomlocomotieven","bevolkingsregistratie volkstellingen","duikboten onderzee\u00ebrs","televisieprogramma's","radioprogramma's","sterilisaties","abortussen","menselijk lichaam","medische behandelingen","kunststromingen","muziekstromingen","rioleringen","waterleiding","onteigeningen","kashba's","verlammingen","spijsvertering","emoties gevoelens","prijsafspraken","kwaliteitsbewaking","pensioenen","lonen salarissen","dierenpensions","stallen","geesteswetenschappen","rubberindustrie","glasindustrie","erfelijkheid","aardwetenschappen geowetenschappen","gewrichten","etymologie letterkunde lingu\u00efstiek literatuurwetenschap taal- en literatuurwetenschap taalkunde taalwetenschap","interne geneeskunde","warenonderzoek","chirurgie","vacatures","arbeidscontracten arbeidsovereenkomsten werkovereenkomsten","fata morgana luchtspiegelingen natuurverschijnselen","reinigingsapparatuur","aardrijkskunde geografie","draagvleugelboten","districtenstelsel kiesstelsels","burgerlijke stand persoonsregistratie","paarden","hanen kippen scharrelkippen","gabbers","punkers","operettes","przewalskipaarden","kaatsen","kabeljauw","kalkoenen","kamelen","kameleons","kanaries","kangoeroes","kappers","karate","karbouwen","karpers","karting skelters","kastanjes","katten poezen","kersen","keurmeesters","kevers torren","kickboksen","kiekendieven","kievieten","kikkers","kinderartsen","kinderrechters","klarinetten","klavecimbels","kleermakers","klokkenluiders","horlogemakers klokkenmakers","klompenmakers","kluten","koala's","koeien","koekoeken","koeten","koks","kolibries","komkommers","konijnen","kool spruitjes","koolzaad","koraal","korfbal","korhoenders","astronauten kosmonauten ruimtevaarders taikonauten","kauwen kraaien","kraanvogels","krabben","krantenbezorgers","kreeften","kruideniers","kuikens","kunstrijden op de schaats kunstschaatsen","kunstzwemmen synchroonzwemmen","kwallen","kwikstaarten","drums marimba's pauken percussie slaginstrumenten trommels","contrabassen oed saz snaarinstrumenten strijkinstrumenten","bandoneons synthesizers toetsinstrumenten","trompetten","kokosnoten","panters","papegaaien","bonen erwten peulvruchten","paddenstoelen zwammen","uien","citrusvruchten sinaasappels","reggae ska","ra\u00ef","kweekscholen pabo pedagogisch onderwijs pedagogische academies","vmbo voorbereidend middelbaar beroepsonderwijs","economisch en administratief onderwijs heao leao meao","huishoudonderwijs huishoudscholen lager huishoud- en nijverheidsonderwijs lhno","noten pinda's","sla","paprika's","neurologische aandoeningen","hersenaandoeningen hersenbloedingen herseninfarcten niet-aangeboren hersenletsel","infectieziekten","beender- en gewrichtsaandoeningen botbreuken osteoporose","allergie\u00ebn","oogaandoeningen staar","verkeersongevallen","ALS amyotrofe lateraal sclerose spierdystrofie spierziekten ziekte van Pompe","BSE gekkekoeienziekte","wintersporten","watersporten","vecht- en verdedigingssporten","denksporten","krachtsporten","mountainbike","jeu de boules","klootschieten","duursporten","luchtsporten vliegen (luchtsporten) vliegsporten","bridge","langlaufen","freestyle snowboarden snowboarden","touwtrekken","zaalvoetbal","lichaamsbeweging","parachutespringen","gras","hoefdieren","hanen hoenderachtigen","vissen","schaal- en schelpdieren schelpdieren","pitbullterri\u00ebrs","politiehonden speurhonden","lammeren","luipaarden","padden","panda's","parkieten","plankton","snippen watersnippen","opgezette dieren","achtervolgingen","arena's","aristocratie","bourgeoisie","kastenstelsel klassenverschillen maatschappelijke verschillen","klassen sociale klassen","bendes jeugdbendes","spinachtigen","huisstofmijten mijten teken","zeezoogdieren","katachtigen","trekvogels vogeltrek","milieubeweging","bergingen","bunkers","decadentie","carri\u00e8re loopbaan","concours hippique","paardendressuur","dresseren","dubbelgangers","duels","gaarkeukens","generatieconflicten","hallucinaties","hel","helden","hemel","inhuldigingen kroningen","haarmode kapsels","kattenkwaad","kinderkoren koren zangkoren","krijgsgevangenen","hoger onderwijs hogescholen tertiair onderwijs","lederindustrie leerlooierijen","luchtafweergeschut luchtdoelgeschut","seksisme","verminkingen","onderdrukking","onderwateropnamen","onzichtbaarheid","overvallen roofovervallen","parlevinkers","persoonsverwisselingen","picknicks","polders","poolgebieden","porselein","processies","filmprojectie","races snelheidswedstrijden","rijtoeren","scheepsongevallen scheepsrampen strandingen","vliegrampen vliegtuigongevallen","spoorwegongevallen treinrampen","uitgaansleven","amusement entertainment showbusiness vermaak","slaapkamers","landingsvaartuigen marineschepen militaire vaartuigen oorlogsschepen","spiegels","fabeldieren fantasiefiguren sprookjesfiguren","steekwapens","tekeningen","toeval","trucage","vampiers","verkeersveiligheid","vrachtwagens","vuurtorens","babysits kinderoppas","keuringen","rijstvelden sawa's","vulkaanuitbarstingen","driehoeksverhoudingen","fotoapparatuur fotocamera's fototoestellen","filmapparatuur","filmmuziek","Latijns-Amerikaanse dansen","Latijns-Amerikaanse muziek","Arabische muziek","raketten","raketten (ruimtevaart) ruimtevaartuigen","ballet","breakdance","buikdansen","walsen","flamenco","tapdansen","afgunst jaloezie","angst vrees","eergevoel","ambitie eerzucht","haat","hebzucht","obsessies","schaamte","schuldgevoel","geluk","ontrouw overspel","verraad","verveling","voyeurisme","wraak","bedrog","verdriet wanhoop","weidevogels","contactloos betalen digitaal betalingsverkeer elektronisch betalen internetbankieren pinnen","betaalpassen creditcards","burenoverlast","buren","landmijnen","zeemijnen","mannenbeweging","Nederlandse records","Europese records","regressietherapie","dance","gewichten maten en gewichten metrologie","nieraandoeningen","r&b","kilometerheffing rekeningrijden","studiehuis","zonderlingen","adviesbureaus organisatie-adviseurs","naslagwerken","atlassen","gsm's mobiele telefonie","moraal normen en waarden waarden en normen","opstanden","gevangenisopstanden","bestralingen radiologie r\u00f6ntgenonderzoek","rechten van het kind","satanisme","sjamanisme","SF sciencefiction","straatartiesten straatmuzikanten","vlas","autokeuringen","belangenverstrengeling","declaraties onkostenvergoedingen","ICT IT communicatietechnologie informatietechnologie","jeugdbladen kinderkranten","lokale politieke partijen","oliemaatschappijen","religieuze conflicten","burn-out burn-outsyndroom","dvd's","gedoogbeleid","globalisering mondialisering","begrotingstekorten financieringsoverschotten financieringstekorten financi\u00ebn overheidsfinanci\u00ebn staatsschuld","vredesmachten vredestroepen","wachtlijsten","waterbeheer watermanagement","getallensymboliek numerologie","gothic","e-mail","internet","appen mobiele berichten sms'en spraakberichten whatsappen","bewindslieden staatssecretarissen","SARS","vogelgriep vogelpest","kindersterfte","mantelzorg","drones onbemande vliegtuigen","achtbanen kermisattracties","mensen met overgewicht","overgewicht","lange mensen","kleine mensen lilliputters","eerwraak","grondverzakking verzakkingen","appartementen","hang-en-sluitwerk sloten (hang-en-sluitwerk)","deuren","ramen","bedrijventerreinen industrieterreinen","eucharistievieringen gebedsdiensten kerkdiensten missen religieuze diensten","filmstudio's","muziekstudio's platenstudio's","radiostudio's","televisiestudio's","opgravingen","straatbeelden straatsc\u00e8nes","stadsbeelden","weg- en waterbouw wegenbouw","opvangcentra wegloophuizen","afkickcentra","alimentatie","anticonceptiepil pil (anticonceptie)","landarbeiders","arbeidsconflicten arbeidsonrust","verpaupering","bedrijfsbeveiliging","demonstraties","bde bijna-doodervaringen","bootvluchtelingen","qat softdrugs","defensiebeleid","derde wereld ontwikkelingslanden","winkeldiefstal","kunstdiefstal kunstroof","sociale dienstplicht","vreemdelingenhaat","paspoorten","economische hulp","dorpsfeesten volksfeesten","oplichting zwendel","sociale controle","geloofsvervolging","bi-culturele relaties gemengde relaties interraciale relaties","gerontologie","geriatrie","gezinshereniging","GHB","hippies","schijnhuwelijken","hersenspoeling indoctrinatie","probleemjongeren randgroepjongeren","werkende jongeren","jeugdwerk vormingswerk","studentenhuisvesting","treinkapingen","vliegtuigkapingen","kindsoldaten","kinderhandel","slavenhandel","mensenhandel","kinderloosheid onvruchtbaarheid","vrouwenmishandeling","kinderpornografie","kinderpostzegels postzegels","kinderprostitutie","kinderziekenhuizen","lijmsnuiven","methadon","militaire parades parades","naakten","ecstasy xtc","aankomst en vertrek vertrek","stewardessen vliegtuigpersoneel","spoorbruggen","ophaalbruggen","havenpersoneel","laden en lossen","landingen start en landing","jachten pleziervaartuigen","ruimtestations","spoorwegmedewerkers spoorwegpersoneel","vissers","vrachtwagenchauffeurs","hoofddoeken","autogebruik automobilisme","luchtvaartincidenten","luchtvaartshows vliegshows","alfabetten schrift","animatiefilms","pamfletten","filmrestauratie","weven","bloopers","baden badkuipen","badkamers","koppelbazen","stiptheidsacties","arbeidsduur arbeidstijdverkorting vierdaagse werkweek vijfdaagse werkweek werktijdverkorting zesdaagse werkweek","begrotingen miljardennota's miljoenennota's overheidsbegrotingen voorjaarsnota's","OPS organisch psychosyndroom","concurrentie marktwerking","consumptie-ijs consumptieijs ijsco's ijsjes","economische crises recessies","equipment tools gebruiksvoorwerpen gereedschappen werktuigen","wereldhandel","kunsthandel","naaimachines","wasmachines","lopende banden","vredesonderhandelingen","caissi\u00e8res winkelpersoneel","muziekindustrie platenindustrie","ziektekostenverzekeringen zorgverzekeringen","spaaracties spaarzegels","personeelstekort","vloeken","blasfemie godslastering","bovennatuurlijke verschijnselen mysteries onverklaarbare verschijnselen","medeleven","goden","engelen","heiligen","jezu\u00efeten","clarissen franciscanen","benedictijnen benedictinessen","pausbezoeken","christelijke feestdagen religieuze feestdagen","vlagverbrandingen","ketters","praatprogramma's talkshows","bioscoopreclame","commercials reclamespots","regie","homo's homoseksuelen lesbiennes","vooroordelen","zonden","minderheden religieuze minderheden","kernafval nucleair afval radioactief afval","naaldbomen","bodemsoorten grondsoorten","bosbranden","jonge dieren","nachtbeelden","schaduwen","nacht","avondbeelden","vliegen","kernincidenten kernrampen","kou vorst","olieverontreiniging","waterstof","hemellichamen","mergel","begroetingen","dagelijks leven","fakirs","ringsteken","grensconflicten grensgeschillen grensincidenten","niet-westerse kunst","paalwoningen","ruilhandel","inspecties wapeninspecties","naamrecht namen persoonsnamen pseudoniemen","hiv-besmetting hiv-positief","helderzienden paragnosten","dierenambulances","lijkschouwingen pathologische anatomie","vertrouwensartsen","neonatologie","hernia rugaandoeningen","doodskisten lijkkisten","komieken","borsten","borstvoeding","doodshoofden gezichten hoofden schedels","brillen","democratisering","maatschappelijke hervormingen","bestuurlijke hervormingen staatkundige hervormingen","griezelen","medische experimenten","handen schudden","omhelzingen","gehandicaptenapparatuur gehandicaptenvoorzieningen gehoorapparaten rollators rolstoelen toegankelijkheidsvoorzieningen","beroepsgeheim geheimhouding","ziekte van Creutzfeldt-Jakob","psychosomatische aandoeningen","vrijlatingen","grafkelders","penningen","borduren","cabaretier cabaretiers conferencier conferenciers stand-up comedian stand-up comedians","culturele manifestaties manifestaties (cultureel)","theaterkostuums","bandleiders orkestleiders","rap","muziekensembles strijkkwartetten","bands popgroepen","ABC-wapens massavernietigingswapens","ADHD Attention Deficit Hyperactivity Disorder","vakantiewerk","bijverdiensten","schooltoetsen","klassen schoolklassen","kunstschilders","leer- en opvoedingsproblemen opvoedingsproblemen","lesmaterialen schoolboeken","onderwijsvernieuwingen","spelling","soennieten","karmelieten karmelitessen","cisterci\u00ebnzers trappisten","augustijnen augustinessen","norbertijnen","psychopaten","popfestivals","filmfestivals","dijkdoorbraken wateroverlast","dierenverblijven","schaapskooien","schaapskudden","herders schaapherders","milieurampen","bioscooppersoneel explicateurs","fitnesscentra sportscholen","kegelen","openluchttheater","amateurtheater","beroepssport topsport","hardlopen","hoogspringen","hordelopen","snelwandelen","steeplechase","verspringen","raceauto's racewagens","baanwielrennen","dierentransport dierenvervoer veevervoer","freestyle ski\u00ebn","aangepaste sport gehandicaptensport parasport","speedbootracen","overheidssteun","prijzen","ritmische gymnastiek","trampolinespringen","sportongevallen","sportincidenten","installatiebedrijven","stoorzenders","betrekkingen bilaterale betrekkingen","verkeersregels","computerinbraak hacken","culturele uitwisselingen interculturele contacten","neofascisme","neonazisme","extremisme","links-extremisme","oorlogsbeelden","georganiseerde misdaad","vechtpartijen","marinebases","laser","identificatieplicht","gevangeniscellen","rechtszittingen","koninginnen","jihad","antikernenergiebeweging","antikernwapenbeweging","antiracisme","antifascisme","antiamerikanisme","anticommunisme","politieke moorden","bankovervallen","treinovervallen","uitleveringen","verkiezingscampagnes","jeugdgevangenissen jeugdinrichtingen justiti\u00eble jeugdinrichtingen tuchtscholen","spreekvaardigheid taalvaardigheid","verkeersovertredingen","sporttrainingen trainingen","toverlantaarns","polsstokhoogspringen","dorpsbeelden","drinkers","eters","eerstesteenleggingen","Hitlergroet fascistengroet","filmfragmenten","filmopnamen radio-opnamen televisieopnamen werkopnamen","hakenkruizen","loven en bieden","meld- en regelkamers","naamborden plaatsnaamborden straatnaamborden","kunstschatten kunstvoorwerpen kunstwerken","paard-en-wagen","zonsondergang zonsondergangen","zonsopgang zonsopgangen","sfeerbeelden","spandoeken","stemmen","toneelfragmenten","bijzondere auto's monstertrucks","bijzondere vaartuigen","bijzondere vliegtuigen","bijzondere voertuigen","schieten","stadsverkeer","Europese eenwording","aandeelhouders","bedrijfsresultaten","bijstand","historische reconstructies re\u00ebnsceneringen","jeugdtheater","kabelomroep kabeltelevisie","kolonisten","levensstijl","culturele prijzen","journalistieke prijzen mediaprijzen radioprijzen televisieprijzen","sportprijzen","wetenschappelijke prijzen","filmprijzen","muziekprijzen","cabaretprijzen theaterprijzen toneelprijzen","literaire prijzen","mensen personen","monumentenzorg","muziekconcoursen","oorlogsverslaggevers","partijcongressen","plastics","politiebureaus","regeringsverklaringen","regeringsvorming","reisverhalen","religieuze kunst","rente","ruimtewandelingen","stripfiguren","gezinszorg thuiszorg","traangas","sportprogramma's","verkeersinformatie","krimi's politieseries","boekenprogramma's literatuurprogramma's","amusementsprogramma's showprogramma's","weekbladen","paardentrams","sloppenwijken","legionairsziekte veteranenziekte","ziekte van Crohn","bouwactiviteiten","justitie","mijnopruiming","bomdemontage","efficiency","collectes inzamelingen","lynchpartijen","steekpartijen","levenswetenschappen","maatschappijwetenschappen","zonen","kleuterleidsters onderwijzers","non-fictie","rococo","reality-tv","loverboys pooierboys","parlementaire onderzoeken","sociaaleconomisch overleg","vergunningen","korte films","reallifesoaps","aansprakelijkheid wettelijke aansprakelijkheid","jeugdzorg","kinderliedjes","mijnenvegers","mijnongevallen mijnrampen","minima","piercing","portretrecht","sekstoerisme","eetstoornissen","industrialisatie","kapingen","bedreigingen","tijd","videoboodschappen","balsemen resomeren uitvaarttechnieken vriesdrogen","windmolens windturbines","contactlenzen","maagaandoeningen maagzweren","sharia","zeebevingen","tsunami's vloedgolven","zwartwerk","dierenhandel","stankoverlast","commandotroepen","isoleercellen","screentests","versterving","e-commerce elektronisch winkelen internethandel teleshopping webwinkels","dierenrechten rechten van het dier","hanengevechten","identificatie","praalwagens","melken","milieudelicten","goederentreinen","kunstveilingen","pauskeuze","rampenoefeningen","websites","medisch personeel zorgberoepen","agrarische beroepen","vrije beroepen","artistieke beroepen kunstenaars","technische beroepen","dienstverlenende beroepen","handelsberoepen","vervoersberoepen","mediaberoepen","wetenschappelijke beroepen","non-profitorganisaties","agrarische ondernemingen","delfstoffenwinning","producten voorwerpen","materialen","plichten","shottermen","sociale bewegingen","sociale vraagstukken","vaardigheden","postorderbedrijven","bedrijfsovernames","forensisch onderzoek","kunstgras","zelfmoordaanslagen","ziekte van Lyme","juridische fouten justiti\u00eble fouten","langeafstandszwemmen","geldtransport waardetransport","kogelslingeren","verwaarlozing","underground","luchtalarm","storingen","tegels","voedselbanken","chatten","punkmuziek","chemotherapie","multimedia","podcasting","terreurbestrijding veiligheidsbeleid","btw","deshormoon","ov-jaarkaart","menstruatie","hindoes","boeddhisten","non-proliferatie","brons","stormschade","stages","biobrandstoffen","autopeds steppen (vervoermiddelen)","idealisme","PTSS posttraumatische stressstoornis","klimaatverandering","alcoholmisbruik drankmisbruik dronkenschap","deconfessionalisering ontkerkelijking secularisatie secularisering","geloofsafval","moties van afkeuring moties van wantrouwen","alawieten","global positioning system gps"]

In [59]:
THRESHOLD = .1

scores = process_manifest("https://static.amsterdamtimemachine.nl/datasprint-toekomsttiendaagse-2025/iiif/saa/2015/03.json", terms)

for image_url, score in scores:
    for term in sorted(score.items(), key=lambda x: x[1])[::-1]:
        if term[1] >= THRESHOLD:
            print(f"{image_url}\t{term[0]}\t{term[1]}")


https://stadsarchiefamsterdam.memorix.io/resources/records/media/09036a82-d31b-77eb-9a7b-5d2acafec4a0/iiif/3/57861915/full/max/0/default.jpg	studentenverenigingen	0.71875
https://stadsarchiefamsterdam.memorix.io/resources/records/media/0a766799-ef46-0d6d-08f3-0884c5d7b7d5/iiif/3/57858384/full/max/0/default.jpg	inspraak medezeggenschap	0.2578125
https://stadsarchiefamsterdam.memorix.io/resources/records/media/150f4af1-e6fb-8cf1-9c50-57041554ff04/iiif/3/57859775/full/max/0/default.jpg	massa menigte mensenmassa	0.2373046875
https://stadsarchiefamsterdam.memorix.io/resources/records/media/170da28f-4b06-1f8b-f153-d512786ac71e/iiif/3/57848906/full/max/0/default.jpg	boodschappen	0.1201171875
https://stadsarchiefamsterdam.memorix.io/resources/records/media/170da28f-4b06-1f8b-f153-d512786ac71e/iiif/3/57848906/full/max/0/default.jpg	bakfietsen	0.10595703125
https://stadsarchiefamsterdam.memorix.io/resources/records/media/2a777a1f-0795-2f46-e74c-9e1a45101472/iiif/3/57855452/full/max/0/default.jpg

studentenverenigingen 0.70703125
